# 0. Imports and Setting up Anthropic API Client

In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [3]:
# Load Prompts and Problem Description
prompt1_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt1_MathematicalModel.txt'
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/MIP/MIP1.txt'

prompt1_file = open(prompt1_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt1 = prompt1_file.read()
print("Prompt 1:\n", prompt1)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1:
 Please write a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 You are a city planner, looking to open facilities at some locations. We have a set of customers and a set of possible locations for opening facilities. Each potential location for establishing a facility incurs a fixed annual activation cost, which must be paid if the facility is used, regardless of the service volume it handles. Furthermore, this service volume at each facility is capped at a maximum annual limit. Additionally, there are transportation costs associated with servicing each customer from each facility.
The goal is to minimize the overall costs, which include both the fixed activation costs for any opened facilities and the transportation

In [4]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 11.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.8 MB/s eta 0:00:00


In [5]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1: Create Mathematical Model

In [6]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt1,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p1 = message.content[0].text

In [7]:
# Print response
print(response_p1)

Thank you for providing the problem description. I will formulate this problem as a mathematical optimization model, including the necessary parameters, decision variables, objective function, and constraints.

Parameters:
- I: set of customers, indexed by i
- J: set of potential facility locations, indexed by j
- f_j: fixed annual activation cost for opening a facility at location j
- c_ij: transportation cost for servicing customer i from facility j
- d_i: annual demand of customer i
- s_j: maximum annual service volume for a facility at location j

Decision Variables:
- x_ij: fraction of customer i's demand serviced by facility j (continuous variable, 0 ≤ x_ij ≤ 1)
- y_j: binary variable indicating whether a facility is opened at location j (y_j = 1 if a facility is opened, 0 otherwise)

Objective Function:
Minimize the total cost, which includes the fixed activation costs for opened facilities and the transportation costs for servicing customers:

minimize ∑_j f_j * y_j + ∑_i ∑_j c

# 2. Prompt 2: Write Code based on Mathematical Model

In [8]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": response_p1}
    ]
)

response_p2 = message2.content[0].text

In [9]:
# Print response
print(response_p2)

Here's a Python code using Pyomo to solve the facility location problem based on the provided mathematical formulation:

```python
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the sets
model.I = pyo.Set(initialize=['A', 'B', 'C', 'D', 'E'])  # Sample customer locations
model.J = pyo.Set(initialize=['1', '2', '3'])  # Sample potential facility locations

# Define the parameters
model.f = pyo.Param(model.J, initialize={'1': 1000, '2': 1500, '3': 1200})  # Sample fixed costs
model.c = pyo.Param(model.I, model.J, initialize={
    ('A', '1'): 10, ('A', '2'): 12, ('A', '3'): 15,
    ('B', '1'): 8, ('B', '2'): 14, ('B', '3'): 11,
    ('C', '1'): 13, ('C', '2'): 9, ('C', '3'): 16,
    ('D', '1'): 7, ('D', '2'): 11, ('D', '3'): 10,
    ('E', '1'): 12, ('E', '2'): 10, ('E', '3'): 14
})  # Sample transportation costs
model.d = pyo.Param(model.I, initialize={'A': 100, 'B': 120, 'C': 80, 'D': 90, 'E': 110})  # Sample demands
model.s = pyo.Param(model.J, init

# 4. Input Problem Data and Test Model Code

In [10]:
# Installing pyomo nd solver
!pip install -q pyomo
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 61.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.9 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [11]:
import pyomo.environ as pyo

# Define the model
model = pyo.ConcreteModel()

# Define the sets
model.I = pyo.Set(initialize=['A', 'B', 'C', 'D', 'E'])  # Sample customer locations
model.J = pyo.Set(initialize=['1', '2', '3'])  # Sample potential facility locations

# Define the parameters
model.f = pyo.Param(model.J, initialize={'1': 1000, '2': 1000, '3': 1000})  # Sample fixed costs
model.c = pyo.Param(model.I, model.J, initialize={
    ('A', '1'): 4, ('A', '2'): 6, ('A', '3'): 9,
    ('B', '1'): 5, ('B', '2'): 4, ('B', '3'): 7,
    ('C', '1'): 6, ('C', '2'): 3, ('C', '3'): 4,
    ('D', '1'): 8, ('D', '2'): 5, ('D', '3'): 3,
    ('E', '1'): 10, ('E', '2'): 8, ('E', '3'): 4
})  # Sample transportation costs
model.d = pyo.Param(model.I, initialize={'A': 80, 'B': 270, 'C': 250, 'D': 160, 'E': 180})  # Sample demands
model.s = pyo.Param(model.J, initialize={'1': 500, '2': 500, '3': 500})  # Sample service volumes

# Define the decision variables
model.x = pyo.Var(model.I, model.J, bounds=(0, 1))  # Fraction of demand serviced
model.y = pyo.Var(model.J, domain=pyo.Binary)  # Facility opening decision

# Define the objective function
model.obj = pyo.Objective(expr=sum(model.f[j] * model.y[j] for j in model.J) +
                          sum(model.c[i, j] * model.x[i, j] * model.d[i] for i in model.I for j in model.J),
                          sense=pyo.minimize)

# Define the constraints
model.demand_constraint = pyo.Constraint(model.I, rule=lambda model, i:
                                         sum(model.x[i, j] for j in model.J) == 1)
model.capacity_constraint = pyo.Constraint(model.J, rule=lambda model, j:
                                           sum(model.x[i, j] * model.d[i] for i in model.I) <= model.s[j] * model.y[j])
model.service_constraint = pyo.Constraint(model.I, model.J, rule=lambda model, i, j:
                                          model.x[i, j] <= model.y[j])

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Optimal Solution:")
print("Objective Value =", pyo.value(model.obj))
for j in model.J:
    if pyo.value(model.y[j]) > 0.5:
        print(f"Facility {j} is opened")
        for i in model.I:
            if pyo.value(model.x[i, j]) > 0:
                print(f"  Customer {i} is serviced by Facility {j} with {pyo.value(model.x[i, j]) * 100}% of demand")

Optimal Solution:
Objective Value = 5609.999999999998
Facility 2 is opened
  Customer A is serviced by Facility 2 with 100.0% of demand
  Customer B is serviced by Facility 2 with 99.9999999999999% of demand
  Customer C is serviced by Facility 2 with 60.0% of demand
Facility 3 is opened
  Customer C is serviced by Facility 3 with 40.0% of demand
  Customer D is serviced by Facility 3 with 100.0% of demand
  Customer E is serviced by Facility 3 with 100.0% of demand


# 5. Correct The Model Code to Test Mathematical Model (if applicable)